In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import sys 
sys.path.append('../../../warpspeed/')
sys.path.append('../../../measutils/')
sys.path.append('../../../pystencils_notebooks')
sys.path.append('..')

%load_ext autoreload
%autoreload 1



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from meas_db import MeasDB

from measured_metrics import MeasuredMetrics, ResultComparer
from predict_metrics import *
from plot_utils import *
import genconv
import json

In [ ]:
meas_db = MeasDB("../bconv.db")
#meas_db.clearDB()

In [ ]:
width = 4096 
height = 4096
input_channels = output_channels = 64
batch_size = 1

In [ ]:



#device = DeviceAmpereA100_80GB()
#device = DeviceMI210()
#device = DeviceAmpereA40()
#device = DeviceHopperH200()
#device = DeviceMI210()



def getDeviceData(device):
    measValues = dict()
    predValues = dict()
    for c_in_per_thread in [1,  2, 4, 8, 16, 32, 64]:
        for x_per_thread in [1, 2 ,4,8,16,32,64]:
            for xblock in [8, 16, 32, 64, 128, 256]:
                for yblock in [1, 2, 4, 8, 16, 32]:
                    for zblock in [1,2,4,8,16,32,64]:
                        if x_per_thread * c_in_per_thread >= 128:
                            continue
                        if xblock * yblock * zblock != 256:
                            continue
                        if (
                            height % x_per_thread != 0
                            or input_channels % c_in_per_thread != 0
                        ):
                            continue

                        if (height % (yblock * x_per_thread) != 0
                            or input_channels % (zblock * c_in_per_thread) != 0
                        ):
                            continue

                        blockSize = (xblock, yblock, zblock)   
                        key = (*blockSize, c_in_per_thread, x_per_thread)


                        date, lc, basic, meas = meas_db.getEntryKeys({"input_channels": input_channels,
                                            "output_channels": output_channels,
                                            "width" : width,
                                            "height" : height,
                                            "batch_size" : batch_size,
                                            "c_in_per_thread" : c_in_per_thread,
                                            "x_per_thread" : x_per_thread,
                                            "xblock" : xblock,
                                            "yblock" : yblock,
                                            "zblock" : zblock,                
                                            "device": '"' + str(device.name) + '"'})



                        if meas is None or basic is None:
                            pass
                            #print(device.name, key)
                        else:
                            metrics = DerivedMetrics(lc, basic, device, meas)
                            measValues[key] = meas
                            predValues[key] = metrics
    return measValues, predValues

devices = [DeviceAmpereA40(), DeviceL40(), DeviceAmpereA100_80GB(), DeviceMI210(), DeviceHopperH200(), DeviceRX6900XT()]

devicePredValues = dict()
deviceMeasValues = dict()

for d in devices:
    deviceMeasValues[d], devicePredValues[d] = getDeviceData(d)
    print(d, len(deviceMeasValues[d]))


                    

In [ ]:
#print(devicePredValues.keys())

device = devices[2]
#print(device)
predValues = devicePredValues[device]
measValues = deviceMeasValues[device]
    


In [ ]:
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator, FixedLocator)

def plotLimiterBars(firstKeys, secondKeys = None, thirdKeys=None):
    

    colors = ["#CC1343", "#349999", "#649903"]

    plt.rcParams.update({'hatch.color': 'w'})


    
    def plotKeys(ax, keys, role = None):
        bottoms = [[0, 0, 0]] * len(keys)
        hatch = 0
        for field in list(predValues[keys[0]].fieldL2LoadV2):
            if field == "total":
                    continue

            fieldMem = [predValues[k].fieldMemLoadV2[field] / predValues[k].lc.flops / device.memBW * device.peakFP32() for k in keys]
            fieldL2 = [predValues[k].fieldL2LoadV2[field] / predValues[k].lc.flops / device.L2BW * device.peakFP32() for k in keys]
            fieldL1 = [predValues[k].fieldL1Cycles[field][2] / predValues[k].lc.flops / (device.smCount * device.clock) * device.peakFP32() for k in keys]

            ax.bar(np.arange(len(keys)) - 0.29, fieldMem, width=0.26, bottom = [b[0] for b in bottoms], color=colors[hatch], label=field, zorder=3 )
            ax.bar(np.arange(len(keys)) - 0.0, fieldL2, width=0.26, bottom = [b[1] for b in bottoms], color=colors[hatch], zorder=3 )
            ax.bar(np.arange(len(keys)) + 0.29, fieldL1, width=0.26, bottom = [b[2] for b in bottoms], color=colors[hatch], zorder=3 )
            bottoms = [[bottoms[i][0] + fieldMem[i], bottoms[i][1] + fieldL2[i], bottoms[i][2] + fieldL1[i]] for i in range(len(bottoms))]
            hatch += 1


        for field in list(predValues[keys[0]].fieldL2Store):
            if field == "total":
                    continue

            fieldMem = [predValues[k].memStoreV1 / predValues[k].lc.flops / device.memBW * device.peakFP32() for k in keys]
            fieldL2 = [predValues[k].fieldL2Store[field] / predValues[k].lc.flops / device.L2BW * device.peakFP32() for k in keys]
            fieldL1 = [predValues[k].fieldL1Cycles[field][2] / predValues[k].lc.flops / (device.smCount * device.clock) * device.peakFP32() for k in keys]


            ax.bar(np.arange(len(keys)) - 0.29, fieldMem, width=0.26, bottom = [b[0] for b in bottoms], color=colors[hatch], label=field, zorder=3 )
            ax.bar(np.arange(len(keys)) - 0.0, fieldL2, width=0.26, bottom = [b[1] for b in bottoms],  color=colors[hatch], zorder=3 )
            ax.bar(np.arange(len(keys)) + 0.29, fieldL1, width=0.26, bottom = [b[2] for b in bottoms],  color=colors[hatch], zorder=3) 
            bottoms = [[bottoms[i][0] + fieldMem[i], bottoms[i][1] + fieldL2[i], bottoms[i][2] + fieldL1[i]] for i in range(len(bottoms))]
            hatch += 1


        #outline
        ax.bar(np.arange(len(keys)) - 0.29, [b[0] for b in bottoms], width=0.26, color="None", edgecolor="k", linewidth=2, zorder=2 )
        ax.bar(np.arange(len(keys)) - 0.0,  [b[1] for b in bottoms], width=0.26, color="None", edgecolor="k", linewidth=2, zorder=2 )
        ax.bar(np.arange(len(keys)) + 0.29, [b[2] for b in bottoms], width=0.26, color="None", edgecolor="k", linewidth=2, zorder=2)


        #actual
        ax.plot(np.arange(len(keys)) - 0.29, [ (perFlop(measValues[k], "memLoad") + perFlop(measValues[k], "memStore")) / device.memBW * device.peakFP32() for k in keys], "o", markersize=3, color="black", zorder=5)
        ax.plot(np.arange(len(keys)) - 0.0, [ (perFlop(measValues[k], "L2Load") + perFlop(measValues[k], "L2Store")) / device.L2BW * device.peakFP32() for k in keys], "o", markersize=3, color="black" , zorder=5)
        if device.name != "MI210":
            ax.plot(np.arange(len(keys)) + 0.29, [ perFlop(measValues[k], "L1TagWavefronts") / (108 * device.clock) * device.peakFP32() for k in keys], "o", markersize=3, color="black", label="measured balance" , zorder=5)

        #performance
        #ax.plot(np.arange(len(keys)), [ device.peakFP32() / measValues[k].tflops / 1000   for k in keys], "_", color="black", markersize=28, label="measured performance", markeredgewidth=2 )
        ax.bar(np.arange(len(keys)), [ device.peakFP32() / measValues[k].tflops / 1000   for k in keys], width=0.89, label="measured performance",  color="#DDDDDD", edgecolor="#AAAAAA" , zorder=1)


        #bar text labels
        for i in range(len(keys)):  
            if bottoms[i][0] / ax.get_ylim()[1] > 0.12 and  bottoms[i][1] / ax.get_ylim()[1] > 0.05 and bottoms[i][2] / ax.get_ylim()[1] > 0.05:
                ax.text(-0.28 + i, 0.024 * ax.get_ylim()[1], "DRAM", color="w", rotation=90, horizontalalignment="center")
                ax.text(0.01 + i, 0.024 * ax.get_ylim()[1], "L2", color="w", rotation=90, horizontalalignment="center")
                ax.text(0.30 + i, 0.024 * ax.get_ylim()[1], "L1", color="w", rotation=90, horizontalalignment="center")
                
            #elif bottoms[i][0] / ax.get_ylim()[1] < 0.12 and  bottoms[i][1] / ax.get_ylim()[1] < 0.05 and bottoms[i][2] / ax.get_ylim()[1] < 0.25:
            #    ax.text(-0.28 + i, 0.1 * ax.get_ylim()[1], "DRAM", color="k", rotation=90, horizontalalignment="center")
            #    ax.text(0.01 + i, 0.1 * ax.get_ylim()[1], "L2", color="k", rotation=90, horizontalalignment="center")
            #    ax.text(0.30 + i, 0.1 * ax.get_ylim()[1], "L1", color="w", rotation=90, horizontalalignment="center")


        ax.axhline(1, color="black")

        ax.set_xticks((np.arange(len(keys))) )
        ax.set_xticklabels([ "{3}x{4}\n{0},{1},{2}".format(*k) for k in keys],  rotation = 30)
 
           
        ax.set_xlim([-0.6,len(keys)-0.4])
        ax.set_ylim([0, ax.get_ylim()[1]])

            
            
 
    def r2f(r):
        return 100 / r

    
    
    if secondKeys is None:
        fig, ax = plt.subplots()
        
        plotKeys(ax, firstKeys)
        yticks = ax.get_yticks() + [1]
        ax.set_ylabel("Ratio: Code Balance / Machine Balance")
        ax.set_yticks(yticks)
 
        ax2 = ax.twinx()        
        ax2.set_ylim(ax.get_ylim())    
        
        ax2.set_ylabel("Percentage of FP32 Peak")
        ax2.set_yticks(yticks)           
        ax2.set_yticklabels([ "{:.0f}%".format(r2f(l)) if l >= 1 else "" for l in yticks] )            
 
        
        ax.minorticks_on()
        ax.xaxis.set_minor_locator(FixedLocator( np.array(ax.get_xticks())+0.5 ))        
        ax.grid(True, "major","y", zorder=-30)        
        ax.grid(True, "minor","x", zorder=-30)
        ax.tick_params(axis='y', which='minor', left=False)
        ax.set_axisbelow(True)

        
        ax.legend()        
    else:
        fig, ax = plt.subplots(1, 3,  gridspec_kw={'width_ratios': [len(firstKeys), len(secondKeys), len(thirdKeys)]})
        plotKeys(ax[0], firstKeys, "first")
        plotKeys(ax[1], secondKeys, "second")
        plotKeys(ax[2], thirdKeys, "second")
        
        
        ylim = max(ax[0].get_ylim()[1], ax[1].get_ylim()[1], ax[2].get_ylim()[1]) * 0.9
        ax[0].set_ylim(0, ylim)        
        ax[1].set_ylim(0, ylim)
        ax[2].set_ylim(0, ylim)
        
        ax[0].set_yticks(ax[0].get_yticks() + [1])
        ax[1].set_yticks(ax[1].get_yticks() + [1])
        ax[2].set_yticks(ax[2].get_yticks() + [1])
        
        plt.minorticks_on()
        
        ax[0].grid(True, "major","y")
        ax[1].grid(True, "major","y")
        ax[2].grid(True, "major","y")
        
        ax[0].grid(True, "minor","x")
        ax[1].grid(True, "minor","x")
        ax[2].grid(True, "minor","x")

        ax[0].set_ylabel("Ratio: Code Balance / Machine Balance")
        #ax[0].set_yticks(yticks)
        
        ax[0].minorticks_on()
        ax[0].xaxis.set_minor_locator(FixedLocator( np.array(ax[0].get_xticks())+0.5 ))        
        ax[0].set_axisbelow(True)
        ax[0].tick_params(axis='y', which='minor', left=False)
        
        ax[1].minorticks_on()
        ax[1].xaxis.set_minor_locator(FixedLocator( np.array(ax[1].get_xticks())+0.5 ))        
        ax[1].set_axisbelow(True)
        ax[1].tick_params(axis='y', which='minor', left=False)

        ax[2].minorticks_on()
        ax[2].xaxis.set_minor_locator(FixedLocator( np.array(ax[1].get_xticks())+0.5 ))        
        ax[2].set_axisbelow(True)
        ax[2].tick_params(axis='y', which='minor', left=False)
        
        ax2 = ax[0].twinx()
        ax2.set_ylim(ax[0].get_ylim())
        ax2.set_yticks(ax[0].get_yticks())
        ax2.set_yticklabels([ "{:.0f}%".format(r2f(l)) if l >= 1 else "" for l in ax2.get_yticks()] )            
      
        
        ax2 = ax[1].twinx()
        ax2.set_ylim(ax[1].get_ylim())
        ax2.set_yticks(ax[1].get_yticks())    
        ax2.set_yticks(ax[1].get_yticks())           
        ax2.set_yticklabels([ "{:.0f}%".format(r2f(l)) if l >= 1 else "" for l in ax2.get_yticks()] )            

        
        ax2 = ax[2].twinx()
        ax2.set_ylim(ax[2].get_ylim())
        ax2.set_yticks(ax[2].get_yticks())
        ax2.set_ylabel("Percentage of FP32 Peak")
        ax2.set_yticks(ax[1].get_yticks())           
        ax2.set_yticklabels([ "{:.0f}%".format(r2f(l)) if l >= 1 else "" for l in ax2.get_yticks()] )            

        
        #ax2.grid(True, "major","y")
        ax[0].tick_params(axis='y', which='minor', left=False)
        ax[1].tick_params(axis='y', which='minor', left=False)
        ax[2].tick_params(axis='y', which='minor', left=False)

        
        #ax[0].set_ylim((0, max(ax[0].get_ylim()[1], ax[1].get_ylim()[1])))
        #ax[1].set_ylim((0, max(ax[0].get_ylim()[1], ax[1].get_ylim()[1])))
    
        

    #fig.set_figwidth(5)
    #fig.set_figheight(5)
    #fig.set_dpi(140)
    
    
    fig.tight_layout()

    return fig,ax
    
    

In [ ]:
keys = [k for k in measValues.keys() if k[0] == 256 and k[1] == 1 and k[3] == 1 and k[4] < 64]
fig,ax = plotLimiterBars( keys )
ax.set_xticklabels([ str(k[4]) for k in keys],  rotation = 0)
ax.set_xlabel("y per thread")
fig.tight_layout()
plt.savefig("./autoplots/limbars_xperthread_" + device.name +  ".svg")
plt.savefig("./autoplots/limbars_xperthread_" + device.name +  ".pdf")



#plt.show()
#plt.close()

In [ ]:
keys = [k for k in measValues.keys() if k[0] == 256 and k[1] == 1 and k[4] == 1 and k[3] < 64]
fig,ax = plotLimiterBars( keys )
ax.set_xticklabels([ str(k[3]) for k in keys],  rotation = 0)
ax.set_xlabel("c_out per thread")
fig.tight_layout()
plt.savefig("./autoplots/limbars_cinperthread_" + device.name +  ".svg")
plt.savefig("./autoplots/limbars_cinperthread_" + device.name +  ".pdf")

In [ ]:
keysBoth = [ (8,32,1, 2, 1)] + [(*k, 2, 1) for k in [(32, 8, 1), (32, 4, 2), (32, 2, 4), (32, 1, 8)]]
print(keysBoth)
keysBlocks1 = [(256,1,1, *k) for k in [(1,1), (1,2), (1,4), (1,8), (1,16), (1,32)]]
keysBlocks2 = [(256,1,1, *k) for k in [(1,1), (2,1), (4,1), (8,1), (16,1), (32,1), (8,8)]]

fig,ax = plotLimiterBars( keysBoth, keysBlocks1, keysBlocks2 )
fig.set_size_inches(14, 5)

#ax.set_ylim(ax.get_ylim()[0], ax.get_ylim()[1] - 4)
#ax.set_xticklabels([ str(k[0:3]) for k in keys],  rotation = 0)
#ax.set_xlabel("c_out per thread")
ax[0].legend()

bbox_props = dict(boxstyle="round", fc="#649903", alpha=0.8, ec="k", lw=1, pad=0.4)
font_props = dict(size = 12, color = "w", fontweight = "bold")

ax[0].text(3, 4.5, "thread block\n shape", ha="center", va="center", rotation=0,
            bbox=bbox_props, **font_props)


bbox_props["boxstyle"] = "rarrow"
ax[1].text(2.8, 6, "Y_PER_THREAD", ha="center", va="center", rotation=0,
            bbox=bbox_props, **font_props)


ax[2].text(3.5, 5, "C_OUT_PER_THREAD", ha="center", va="center", rotation=0,
            bbox=bbox_props, **font_props)

ax[2].text(6, 3, "both", ha="center", va="center", rotation=-90,
            bbox=bbox_props, **font_props)

ax[2].text(0.7, 0.9, device.getDisplayName(), fontsize=24, horizontalalignment='left', fontweight="bold",
     verticalalignment='center', color="k", transform = ax[2].transAxes, backgroundcolor="w")




fig.tight_layout()
plt.savefig("./autoplots/limbars_both_" + device.name +  ".svg")
plt.savefig("./autoplots/limbars_both_" + device.name +  ".pdf")

In [ ]:
from matplotlib.ticker import MaxNLocator

colors = ["#CC1343", "#349999", "#649903", "#FFAA11"]

def polarPlot(keys, hlKeys, measValues, predValues, device):   
    
    def getData(keys):
        data = [ [ predValues[k].memLoadV4 / device.memBW / predValues[k].lc.flops,
                   predValues[k].L2LoadV2 / device.L2BW / predValues[k].lc.flops,
                   predValues[k].L1Cycles / (device.smCount * device.clock) / predValues[k].lc.flops] for k in keys if k in predValues] 

        print(device.name)
        print(device.peakFP32())

        print(predValues[list(predValues.keys())[0]].L1Cycles)
        
        data = [[ v * device.peakFP32() for v in d ] for d in data]
        data = [ [*d , d[0]] for d in data ]
        return data
    
    data = getData(keys)
    hlData = getData(hlKeys)
        
    
    categories = 3


    label_loc = np.linspace(start=0, stop=2 * np.pi, num=categories + 1)


    plt.figure(figsize=(4, 4), facecolor="white", dpi=150)
    ax=plt.subplot(polar=True)
    for i in range(len(data)):
        ax.plot(label_loc, data[i], '.-', color="black", alpha=0.08, linewidth=3, markersize=18, markeredgewidth=0)

    ax.axhline(1, color="white", linewidth=4).get_path()._interpolation_steps = 180
    for i in range(len(hlData)):
        ax.plot(label_loc, hlData[i], '-', label= "({},{},{}) {}x{}".format(*hlKeys[i]), color=colors[i], linewidth=2)
   
    # DeviceName
    ax.text(0.07, 0.85, device.getDisplayName(), fontsize=18, horizontalalignment='left', fontweight="bold",
     verticalalignment='center', color="k", transform = ax.transAxes, backgroundcolor="w")


    
    # Fix axis to go in the right order and start at 12 o'clock.
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)

    # Draw axis lines for each angle and label.
    ax.set_thetagrids(np.degrees(label_loc), ["DRAM", "L2", "L1", "DRAM"])
  

    # Ensure radar goes from 0 to 100. it also removes the extra line
    #ax.set_ylim(0, 5)
    # You can also set gridlines manually like this:
    # ax.set_rgrids([20, 40, 60, 80, 100])

    # Set position of y-labels (0-100) to be in the middle
    # of the first two axes.
    ax.set_rlabel_position(180 / 4)

    # Make the y-axis (0-100) labels smaller.
    #ax.tick_params(axis='y', labelsize=8)
    # Change the color of the circular gridlines.
    #ax.grid(color='#AAAAAA')
    # Change the color of the outermost gridline (the spine).
    #ax.spines['polar'].set_color('#eaeaea')
    # Change the background color inside the circle itself.
    ax.set_facecolor('#FAFAFA')

    ax.yaxis.set_major_locator(MaxNLocator(integer=True))


    ax.axhline(1, color="black").get_path()._interpolation_steps = 180

    #ax.set_yscale("log")

    # Add a legend as well.
    ax.legend(loc='lower center') #, bbox_to_anchor=(1.3, 1.1))
    
    return ax

#keys = [k for k in measValues.keys() if k[2] > 8 and k[4] < 32 and k[3] < 32]
#fig, ax = polarPlot(keys, [(64, 1, 4, 1, 1), (64, 1, 4, 2, 2), (64, 1, 4, 4, 4), (64, 1, 4, 8, 8)], measValues, predValues, device)

for d in devices:

    keys = [k for k in deviceMeasValues[d].keys() if k[0] > 16 and k[2] > 0 ]
    ax = polarPlot(keys, [(256, 1, 1, 1, 1), (256, 1, 1, 2, 2), (256, 1, 1, 4, 4), (256, 1, 1, 8, 8)], deviceMeasValues[d], devicePredValues[d], d)

    plt.subplots_adjust(left=0.0, right=1.0, top=1.0, bottom=0.0)
    plt.savefig("./autoplots/polarplot_" + d.name +  ".pdf")
    plt.savefig("./autoplots/polarplot_" + d.name +  ".svg")
    plt.show()

In [ ]:

volumeScatterPlot([((k[0], k[1], k[2]), perFlop(measValues[k], "memLoad"), perFlop(predValues[k], "memLoadV1"), k[3]) for k in measValues], "Memory Load Volumes V1 " + str(device.name))
volumeScatterPlot([((k[0], k[1], k[2]), perFlop(measValues[k], "memLoad"), perFlop(predValues[k], "memLoadV2"), k[3], perFlop(predValues[k],"memLoadV1")) for k in measValues], "Memory Load Volumes V2 " + str(device.name))
volumeScatterPlot([((k[0], k[1], k[2]), perFlop(measValues[k], "memLoad"), perFlop(predValues[k], "memLoadV3"), k[3], perFlop(predValues[k],"memLoadV1")) for k in measValues], "Memory Load Volumes V3 " + str(device.name))
volumeScatterPlot([((k[0], k[1], k[2]), perFlop(measValues[k], "memLoad"), perFlop(predValues[k], "memLoadV4"), k[3], perFlop(predValues[k],"memLoadV3")) for k in measValues], "Memory Load Volumes V4 " + str(device.name))


In [ ]:
def fitValues(measValues, predValues):
    fig, ax = plt.subplots()
    fig.set_figwidth(5)
    fig.set_figheight(5)
    fig.set_dpi(140)

    xdata = []
    ydata = []
        


    meas = measValues
    pred = predValues
    for a in [0,1]:
        keys = [k for k in meas if (pred[k].memLoadV1 - meas[k].memLoad) > 0.3 and
               (pred[k].basic.waveMemLoadOverlap[a] / pred[k].basic.waveValidCells) > 0.3]
        x = [ max(0.0, min(40.0, device.sizeL2 / (pred[k].basic.waveMemOld[a] + pred[k].basic.waveMemLoadNew - pred[k].basic.waveMemLoadOverlap[a]))) for k in keys]
        y = [ max(0.0, min(1.0, (pred[k].memLoadV1 - meas[k].memLoad) / (pred[k].basic.waveMemLoadOverlap[a] / pred[k].basic.waveValidCells / pred[k].lc.blocking_factors[1] / pred[k].lc.blocking_factors[2])))  for k in keys]
        ax.plot(x, y, ".", alpha=0.2)
        xdata.extend(x)
        ydata.extend(y)

    values = np.arange(0.0, 10.0, 0.1)
    
    #if len(keys) == 0:
    #    return
     
    from scipy.optimize import curve_fit

    def func(x, a, b, c):
        return a * np.exp(-b*np.exp(-c*x))


    popt, pcov = curve_fit(func, xdata, ydata, bounds=([0.999999, -np.inf, -np.inf], [1.0, np.inf, np.inf]), maxfev=20000)
    print(popt)
    print(pcov)

    xdata = np.array([*list(xdata), *[  i / 155 for i in range(1,155) ]])
    xdata.sort()

    
    plt.plot(xdata, func(xdata, *popt), 'r-',
             label='fit: a=%5.3f, \n     b=%5.3f,\n     c=%5.3f' % tuple(popt))

    ax.axvline(1.0)
    ax.legend()

fitValues(measValues, predValues)

In [ ]:
volumeScatterPlot([(k[0:3], perFlop(measValues[k], "memStore"), perFlop(predValues[k], "memStoreV1"), k[3]) for k in measValues], "Memory Store Volumes V1 " + str(device.name))
volumeScatterPlot([(k[0:3], perFlop(measValues[k], "memStore"), perFlop(predValues[k], "memStoreV2"), k[3], perFlop(predValues[k], "memStoreV1"), ) for k in measValues], "Memory Store Volumes V2 " + str(device.name))


In [ ]:
volumeScatterPlot([(k[0:3], perFlop(measValues[k], "L2Load_tex"), perFlop(predValues[k], "L2LoadV1"), k[3]) for k in measValues], "Stencil L2 Load Volumes V1  " + str(device.name))
volumeScatterPlot([(k[0:3], perFlop(measValues[k], "L2Load_tex"), perFlop(predValues[k], "L2LoadV2"), k[3], perFlop(predValues[k], "L2LoadV1")) for k in measValues], "Stencil L2 Load Volumes V2 " + str(device.name))
volumeScatterPlot([(k[0:3], perLup(measValues[k], "L2Store"), perLup(predValues[k], "L2Store"), k[3] ) for k in measValues], "Stencil L2 Store Volumes " + str(device.name))



In [ ]:
fig, ax = volumeScatterPlot([(k[0:3], perFlop(measValues[k], "L1DataPipeWavefronts"), perFlop(predValues[k], "L1DataPipeCycles") + perFlop(predValues[k], "smL1Alloc") / 512, k[4]) for k in measValues], "L1 Data Pipe Cycles  " + str(device.name))
fig, ax = volumeScatterPlot([(k[0:3], perFlop(measValues[k], "L1TagWavefronts"), perFlop(predValues[k], "L1TagCycles") , k[4]) for k in measValues], "L1 Tage Wavefront Cycles  " + str(device.name))
fig, ax = volumeScatterPlot([(k[0:3], max( perFlop(measValues[k], "L1DataPipeWavefronts"), perFlop(measValues[k], "L1TagWavefronts")),
                                      max( perFlop(predValues[k], "L1TagCycles"), perFlop(predValues[k],"L1DataPipeCycles") + perFlop(predValues[k], "smL1Alloc") / 512 ), k[4]) for k in measValues], "L1 Cache Cycles  " + str(device.name))
fig, ax = volumeScatterPlot([(k[0:3], measValues[k].L1DataPipeWavefronts, measValues[k].L1TagWavefronts  , k[4]) for k in measValues], "L1 Tag vs Datapipe Cycles  " + str(device.name))
ax.set_xlabel("dataPipeWavefronts")
ax.set_ylabel("tagWavefronts")
for k in sorted(list(predValues), key = lambda p : max( measValues[p].L1TagWavefronts, measValues[p].L1DataPipeWavefronts))[-7:]:
    print("{:17s}  {:7.2f} {:7.2f} : {:7.2f} {:7.2f} {:7.2f}".format( str(k), measValues[k].L1TagWavefronts, measValues[k].L1DataPipeWavefronts, predValues[k].L1TagCycles, predValues[k].L1DataPipeCycles, predValues[k].L1DataPipeCycles + predValues[k].smL1Alloc / 512 ))

In [ ]:
categories = ["flops", "L1", "L2", "DRAM" ]
r = 0
keys = measValues.keys()
volumeScatterPlot([(k[0:3], measValues[k].tflops, predValues[k].perfTFlopsV1, categories[predValues[k].limV1]) for k in keys], " Extended Roofline V1,  " + str(device.name))

volumeScatterPlot([(k[0:3], measValues[k].tflops, predValues[k].perfTFlopsV2, categories[predValues[k].limV2], predValues[k].perfTFlopsV1) for k in keys], "R  Extended Roofline V2,  " + str(device.name))
fig, ax = volumeScatterPlot([( k[0:3], measValues[k].tflops, predValues[k].perfTFlopsV3, categories[predValues[k].limV3], predValues[k].perfTFlopsV2) for k in keys], " Extended Roofline V3,  " + str(device.name))
ax.set_xlabel("measured performance, GFlop/s")
ax.set_ylabel("predicted performance, GFlop/s")
plt.savefig("./autoplots/extroofline_v3_range" + str(3) + "_" + device.name +  ".svg")
#volumeScatterPlot([(k[0:3], measValues[k].tflops, predValues[k].perfTFlopsV4, categories[predValues[k].limV4], predValues[k].perfTFlopsV3) for k in keys], " Extended Roofline V4,  " + str(device.name))



fig, ax = volumeScatterPlot([(k[0:3], measValues[k].tflops, predValues[k].perfTFlopsPheno, categories[predValues[k].limPheno], predValues[k].perfTFlopsV2) for k in keys], "R" + str(r) + " Extended Roofline Pheno,  " + str(device.name))
ax.set_xlabel("measured performance, GFlops/s")
ax.set_ylabel("predicted performance, GFLops/s")
plt.savefig("./autoplots/extroofline_pheno_range" + str(3) + "_" + device.name +  ".svg")

    
volumeScatterPlot([(k[0:3], measValues[k].tflops, predValues[k].perfTFlopsEPMPheno, categories[predValues[k].limPheno], predValues[k].perfTFlopsPheno) for k in keys], "R" + str(r) + " EPM Pheno,  " + str(device.name))
#volumeScatterPlot([(k[0:3], measValues[k].tflops, predValues[k].perfTFlops2LimV4, categories[predValues[k].lim2LimPheno], predValues[k].perfTFlopsPheno) for k in keys], "R" + str(r) + " Standard Roofline Pheno,  " + str(device.name))


In [ ]:
predTop = [(predValues[k].perfTFlopsEPMPheno, k[0:3], k[3], k[4]) for k in measValues.keys() ]
measTop = [(measValues[k].tflops,  k[0:3], k[3], k[4]) for k in measValues.keys() ]

print(len(predTop))

predTop = sorted(predTop)[-22:]
measTop = sorted(measTop)[-22:]


print("Top Preds:")
for p in predTop:    
    print("{} {:5.2f} {!r:12} {!r:4} {!r:4}".format("**" if p[1:] == measTop[-1][1:] else "  ", *p))

print()
print("Top Meas")
for p in measTop:    
    print("{} {:5.2f} {!r:12} {!r:4} {!r:4}".format("**" if p[1:] == predTop[-1][1:] else "  ", *p))

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5), dpi=200)



balances = []

pMaxMax = 0

def plotLimiter(q1, q2, color):    
    machineBalance = device.peakFP32() / getattr(device, q2) *0.8
    codeBalance = [ 1 / sum([perFlop(measValues[k], m) for m in q1]) for k in measValues]
    print(machineBalance)
    
    p1 = min([measValues[k].tflops for k in measValues])  / getattr(device, q2) * 1000
    
    p2 = max(codeBalance)
    plt.plot( (  machineBalance, p2), (device.peakFP32() / 1000, device.peakFP32() / 1000), color="gray" )
    plt.plot(  (0, machineBalance), (0 * getattr(device, q2) / 1000, device.peakFP32() / 1000), color=color , solid_capstyle="round" )
    plt.scatter( codeBalance, [measValues[k].tflops for k in measValues], color=color, marker=["v", "s", "^"][plotLimiter.marker], alpha=0.6 )
    plotLimiter.marker += 1
    return codeBalance

plotLimiter.marker = 0    
    
balances.append( plotLimiter( ["memLoad", "memStore"], "memBW", "C0") )
balances.append( plotLimiter(["L2Load_tex", "L2Store"], "L2BW", "C1") )


peakRate = device.smCount * device.clock * 64
machineBalance = device.peakFP32() / peakRate

codeBalance = [1 /  perFlop(predValues[k], "L1Cycles") / 64 for k in measValues]
p1 = min([measValues[k].tflops for k in measValues]) / peakRate * 1000
p2 = max(codeBalance)

print(p1)

plt.plot( (  machineBalance, 20), (device.peakFP32() / 1000, device.peakFP32() / 1000), color="gray", solid_capstyle="butt" )
plt.plot(  (0, machineBalance), (0 * peakRate  / 1000, device.peakFP32() / 1000), color="C2" , solid_capstyle="round"  )
plt.scatter( codeBalance, [measValues[k].tflops for k in measValues], color="C2", alpha=0.6  )

balances.append( codeBalance )


for k in range(len(balances[0])):
    plt.plot( (balances[0][k], balances[1][k]), [ measValues[ list(measValues.keys())[k] ].tflops ] * 2, color= getColor(list(measValues.keys())[k][0:3]), zorder=-1, alpha=0.2  )
    plt.plot( (balances[1][k], balances[2][k]), [ measValues[ list(measValues.keys())[k] ].tflops ] * 2, color=getColor(list(measValues.keys())[k][0:3]), zorder=-1, alpha=0.2  )

ax.set_xscale("log")
ax.set_yscale("log")

ax.set_ylim([ 0.1, 26])
ax.set_xlim([ 0.05, 206])

ax.set_xlabel("arithmetic Intensity, Flop/B or Flop/cycle")
ax.set_ylabel("Performance, GFlop/s")

fig.tight_layout()
fig.savefig("autoplots/bconv_roofline_1x1_" + device.name + ".pdf")


In [ ]:
from math import log, exp

def L2Latency(wdev, payload, clock):

    # return max(200, (payload * wdev) / 1500 * clock)
    x = wdev * payload / 8 / 32
    # lat = 210 + 102* log(1 + exp((x-1400)*0.00138))
    lat = max(237, 175 + 440 * log(1 + exp((x - 1000) * 0.0002)))
    # print("{:6.0f} {:6.0f} {:6.1f}".format(x, lat, payload / 8 / 32))
    return lat


def memLatency(wdev, payload, clock):
    # return max(250, wdev*payload / 780 * clock)
    x = wdev * payload / 8 / 32
    lat = max(210, 0 + 1300 * log(1 + exp((x - 1000) * 0.00017)))
    # print("{:6.0f} {:6.0f} {:6.1f}".format(x, lat, payload / 8 / 32))
    return lat


for x in range(1, 108):
    print(memLatency(x * 2048, 100, 1.41))
    print( max(210, x * 2048 * 100 / 1600e9 * 1.41e9))
    print()


In [ ]:
from random import *
from math import sqrt

nclusters = min(4, len(measValues.keys() ) // 2)

keys = list(measValues.keys())
shuffle(keys)


clusters = [[] for n in range(nclusters)]
clusterCenters = [keys[i] for i in range(nclusters)]

dim1 = { key : perFlop(measValues[key], "L2Load") for key in measValues.keys()}
dim2 = { key : (perFlop(predValues[key], "L2LoadV2") - perFlop(measValues[key], "L2Load"))*4 for key in measValues.keys()}



def distance(key1, key2):
    return sqrt( (dim1[key1] - dim1[key2])**2 +
                 (dim2[key1] - dim2[key2]) **2 ) 
    

for i in range(0,20):

    for c in range(nclusters):
        print(len(clusters[c]))
        if len(clusters[c]) <= 1 or randint(0,2000) == 0:
            clusterCenters[c] = choice(keys)

    clusters = [[] for n in range(nclusters)]    
    for key in keys:
        minDistance = -1
        minCluster = -1
        for c in range(nclusters):            
            d = distance(key, clusterCenters[c])

            if d < minDistance or minDistance < 0:
                minDistance = d
                minCluster = c
        clusters[minCluster].append(key)
            
    for c in range(nclusters):
        shuffle(clusters[c])
        lowestDistanceSum = -1
        lowestKey = -1
        for k1 in clusters[c]:
            distanceSum = 0
            for k2 in clusters[c]:
                distanceSum += distance(k1, k2)**2
            if distanceSum < lowestDistanceSum or lowestDistanceSum < 0:
                lowestDistanceSum = distanceSum
                lowestKey = k1
        if c != -1:
            clusterCenters[c] = lowestKey
        
            
  
        
    print(clusterCenters)
          
    keyClusters = {}
    for k in keys:
        for c in range(nclusters):
            if k in clusters[c]:
                keyClusters[k] = c

    fig, ax = plt.subplots(figsize=[5, 5], dpi=200)
    #volumeScatterPlot([(k[0:3], perFlop(measValues[k], "memLoad"), perFlop(measValues[k], "L2Load_tex"), keyClusters[k]) for k in keys], "test_" + str(device.name))    
    for c in range(nclusters):
        ax.scatter( [dim1[k] for k in clusters[c] ], [dim2[k] for k in clusters[c]])
        if clusterCenters[c] != -1:
            ax.plot( dim1[clusterCenters[c]],  dim2[clusterCenters[c]], "+", markersize=15, color="black" )           
            ax.text(dim1[clusterCenters[c]] - 0.01,  dim2[clusterCenters[c]] + (random())/100 , str( clusterCenters[c]), fontsize=7)

    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    #ax.set_xlim(min(xlim[0], ylim[0]), max(xlim[1], ylim[1])  )
    #ax.set_ylim(min(xlim[0], ylim[0]), max(xlim[1], ylim[1])  )
        
    plt.show()
    plt.close(fig)



